In [3]:
search_for = 128
start_from = 0
threads = 10
thread = 6
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.017093420028686523
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 330
all cases: 866251507
type: [1, 1, 1, 1, 128] 128
cases of this type: 268435456
100000 34.78418934016455
200000 60.57578591593346
300000 68.9174158365345
400000 68.80610897449573
500000 67.76779983187417
600000 71.0397649957696
700000 67.51002738386782
800000 67.0893448182814
900000 67.40758857608252
1000000 66.40355970914658
1100000 19.94228621599607
1200000 68.03075680156807
1300000 66.16977661528254
1400000 66.40546491617224
1500000 66.34815914847772
1600000 70.17285980684154
1700000 67.86903159862688
1800000 66.9288815945044
1900000 67.61783331901941
2000000 66.62893243169108
2100000 52.25586851004837
2200000 35.58390887502648
2300000 63.76156392982247
2400000 64.2140987725615
2500000 62.6723950553965
2600000 62.974667984466635
2700000 65.64906574897867
2800000 63.10026238722819
2900000 62.494841690051096
3000000 62.21637521646845
3100

30100000 77.11258693401868
30200000 78.50856785998437
30300000 69.53894047883351
30400000 58.29629801055814
30500000 38.45152663830765
30600000 68.16763215738256
30700000 72.08498233233664
30800000 70.41481768776494
30900000 74.64451174210802
31000000 77.55110466656245
31100000 80.50360469800108
31200000 70.66260789800545
31300000 37.05917616045699
31400000 74.53816550224106
31500000 80.66197985977442
31600000 72.63680272362024
31700000 57.8832670984726
31800000 47.80974426457976
31900000 74.93386298653515
32000000 74.22841878830828
32100000 72.16949759029873
32200000 67.01887473150425
32300000 69.7923668050124
32400000 67.1923172335338
32500000 54.571388468967314
32600000 39.813514672382
32700000 62.85907070943751
32800000 64.96854633379387
32900000 68.18035913952315
33000000 69.75209033223872
33100000 74.2951686731044
33200000 75.80079462422883
33300000 60.981220804352866
33400000 49.338292704632536
33500000 74.84893825650343
33600000 80.22747122236319
33700000 79.80290549457283
3380

60600000 76.55141469265116
60700000 76.98871392378169
60800000 78.4872524086499
60900000 75.07649221531102
61000000 73.9847370920018
61100000 71.61817605709457
61200000 76.57684817791436
61300000 51.44698744855317
61400000 50.278521678605046
61500000 68.35027324120328
61600000 64.91960821698659
61700000 69.27622586157226
61800000 63.88803333905672
61900000 30.817196585412074
62000000 59.16035479786137
62100000 68.07604025503676
62200000 64.46260459179291
62300000 63.79904412417156
62400000 65.50338811629015
62500000 37.08566347949701
62600000 69.24508675075123
62700000 77.12352339000958
62800000 71.98723525062184
62900000 79.42452340396115
63000000 79.57394282408475
63100000 77.9409319451977
63200000 74.31956116650478
63300000 79.51335923708314
63400000 63.90896300089189
63500000 42.05668194384988
63600000 66.36160067046247
63700000 71.14527750835524
63800000 72.43406001557597
63900000 69.36600784921232
64000000 29.403699378139176
64100000 63.51215078042151
64200000 64.8892789708876
64

91100000 60.8658912114926
91200000 59.905661885132574
91300000 24.85515241089965
91400000 62.59557614277429
91500000 65.19108246303344
91600000 37.48739883996158
91700000 56.23760940792749
91800000 62.064436512727525
91900000 67.27021246641677
92000000 64.68062569133971
92100000 74.60757164365958
92200000 69.09709508232467
92300000 69.61513631312089
92400000 75.4660776886261
92500000 72.86905177025703
92600000 75.14125699237202
92700000 70.23608225388239
92800000 70.40346451835141
92900000 64.17420331655322
93000000 48.58304584093003
93100000 44.60798289340601
93200000 65.93067564551541
93300000 58.49704959005475
93400000 26.092099461412033
93500000 69.62828666944158
93600000 62.870494529845935
93700000 29.908308532423014
93800000 70.20684078529223
93900000 65.15505253876165
94000000 64.93284716274135
94100000 68.48929388800936
94200000 71.45134030966528
94300000 71.10032352825738
94400000 69.7406236685555
94500000 67.58578238607836
94600000 68.72015095647293
94700000 67.09736973860426

120700000 31.912001191211957
120800000 42.850287059166924
120900000 58.1557526618192
121000000 57.83592713196085
121100000 63.695875472891295
121200000 60.088052059511654
121300000 63.759206747426965
121400000 59.55853357418864
121500000 66.2994022978133
121600000 65.14428818341261
121700000 69.95684343201
121800000 66.49653604239855
121900000 65.83959548124896
122000000 63.89705601567138
122100000 61.58860371324367
122200000 67.25066225279225
122300000 61.04544380577567
122400000 60.51359801304365
122500000 60.02089250935217
122600000 41.71293481138287
122700000 22.857306245108774
122800000 24.87503992887259
122900000 49.945721356753225
123000000 62.085498699183745
123100000 61.19610050753511
123200000 61.725378258787764
123300000 63.02469738235759
123400000 62.23940870520706
123500000 61.19327918192011
123600000 66.08556886887303
123700000 67.55868648877714
123800000 69.19985217624672
123900000 59.8795131431329
124000000 57.853356793496076
124100000 55.94093889288548
124200000 59.109

...

best so far: 0
type: [1, 2, 16, 1, 4] 128
cases of this type: 131072
799800000 0.24455893202792472
799900000 0.2676350404607991
best so far: 0
type: [1, 2, 16, 2, 2] 128
cases of this type: 65536
best so far: 0
type: [1, 2, 16, 4, 1] 128
cases of this type: 32768
800000000 0.2425066243966884
best so far: 0
type: [1, 2, 32, 1, 2] 128
cases of this type: 32768
best so far: 0
type: [1, 2, 32, 2, 1] 128
cases of this type: 16384
best so far: 0
type: [1, 2, 64, 1, 1] 128
cases of this type: 8192
best so far: 0
type: [1, 4, 1, 1, 32] 128
cases of this type: 4194304
800100000 0.5033531128028651
800200000 1.3365423177846274
800300000 1.1141904444088617
800400000 1.044319591312868
800500000 1.22069700426144
800600000 1.4735929373657894
800700000 0.9483074933031215
800800000 0.8648113276357948
800900000 1.0853002521040505
801000000 1.424228160168393
801100000 1.37311459068381
801200000 0.9997067777458456
801300000 1.0826810915345284
801400000 1.0280401539281487
801500000 0.8839910147338245
80160

813400000 1.4775890601138215
813500000 0.9880158821033074
813600000 1.2750979284815809
813700000 1.293345011123375
813800000 1.0763133123827164
813900000 0.9913352927974616
814000000 1.3193262121138574
814100000 1.2808034254364769
814200000 0.9731566733164575
814300000 0.9935347920750877
814400000 0.8837018236460176
814500000 0.921232278299349
814600000 0.9908362256713172
814700000 1.0609923738341211
814800000 1.2766556985255493
814900000 0.9838533062202156
815000000 0.9129235645162251
815100000 0.9053073463068253
815200000 1.0806764309677959
815300000 0.9926223786710091
815400000 1.6125519804669843
815500000 1.3367177495075284
815600000 1.6818000927131949
815700000 1.067120897015102
815800000 0.7867429864400088
815900000 1.242914814628605
816000000 0.9461163356140534
816100000 0.8348891317291259
816200000 0.9796756025048553
816300000 0.9139457228354063
816400000 0.7188602851441344
816500000 1.1461481934390234
816600000 1.1895128024876058
816700000 1.0757538529836594
816800000 1.525512

841100000 0.2565132348930736
841200000 0.22419284563705524
841300000 0.1804764594953497
841400000 0.24936884824639094
841500000 0.17854305677699644
841600000 0.22876269031127494
841700000 0.24344445664256345
841800000 0.15488984060299926
best so far: 0
type: [2, 1, 1, 8, 8] 128
cases of this type: 2097152
841900000 0.26370228406794666
842000000 0.21118871246334575
842100000 0.17758145271049872
842200000 0.29253913210027216
842300000 0.3416089889998906
842400000 0.21220799245877003
842500000 0.23034820760133465
842600000 0.16905262955500153
842700000 0.24408547897588018
842800000 0.19280701022382976
842900000 0.18707501638246443
843000000 0.23910540832234092
843100000 0.16951375806379915
843200000 0.21187299129972129
843300000 0.20293088720102576
843400000 0.16494399272405105
843500000 0.2395371518714878
843600000 0.17438501542080442
843700000 0.1928312802817583
843800000 0.22164203733331891
843900000 0.1565018772840937
best so far: 0
type: [2, 1, 1, 16, 4] 128
cases of this type: 10485

861200000 0.02026195186151266
861300000 0.03747424006774558
best so far: 0
type: [2, 2, 2, 2, 8] 128
cases of this type: 262144
861400000 0.03847378051260511
861500000 0.03186106664962106
861600000 0.01853854574312369
best so far: 0
type: [2, 2, 2, 4, 4] 128
cases of this type: 131072
861700000 0.028497604943972823
best so far: 0
type: [2, 2, 2, 8, 2] 128
cases of this type: 65536
861800000 0.0222752349024064
best so far: 0
type: [2, 2, 2, 16, 1] 128
cases of this type: 32768
best so far: 0
type: [2, 2, 4, 1, 8] 128
cases of this type: 131072
861900000 0.021762684359641873
best so far: 0
type: [2, 2, 4, 2, 4] 128
cases of this type: 65536
862000000 0.020523209832197426
best so far: 0
type: [2, 2, 4, 4, 2] 128
cases of this type: 32768
best so far: 0
type: [2, 2, 4, 8, 1] 128
cases of this type: 16384
862100000 0.01684960105731487
best so far: 0
type: [2, 2, 8, 1, 4] 128
cases of this type: 32768
best so far: 0
type: [2, 2, 8, 2, 2] 128
cases of this type: 16384
best so far: 0
type: [2,

866100000 0.0007591241509351466
best so far: 0
type: [8, 1, 1, 2, 8] 128
cases of this type: 32768
best so far: 0
type: [8, 1, 1, 4, 4] 128
cases of this type: 16384
best so far: 0
type: [8, 1, 1, 8, 2] 128
cases of this type: 8192
best so far: 0
type: [8, 1, 1, 16, 1] 128
cases of this type: 4096
best so far: 0
type: [8, 1, 2, 1, 8] 128
cases of this type: 16384
best so far: 0
type: [8, 1, 2, 2, 4] 128
cases of this type: 8192
866200000 0.00024373416846990584
best so far: 0
type: [8, 1, 2, 4, 2] 128
cases of this type: 4096
best so far: 0
type: [8, 1, 2, 8, 1] 128
cases of this type: 2048
best so far: 0
type: [8, 1, 4, 1, 4] 128
cases of this type: 4096
best so far: 0
type: [8, 1, 4, 2, 2] 128
cases of this type: 2048
best so far: 0
type: [8, 1, 4, 4, 1] 128
cases of this type: 1024
best so far: 0
type: [8, 1, 8, 1, 2] 128
cases of this type: 1024
best so far: 0
type: [8, 1, 8, 2, 1] 128
cases of this type: 512
best so far: 0
type: [8, 1, 16, 1, 1] 128
cases of this type: 256
best so 